In [68]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import re
import math
import numpy
import pandas as pd 

import json
import csv
from itertools import chain
from typing import List
from multiprocessing.dummy import Pool as ThreadPool 



def read_df(city_abbr):
    data = pd.read_csv(f'../hh-data/tables/{city_abbr}/fulltime/url-adress-salary.csv')
    return data


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

def get_lat_lon(adress):
    api_key = '04bce1f4-b257-45ac-acf5-673a9e894fc1'
    api_url = f'https://geocode-maps.yandex.ru/1.x/?apikey={api_key}&format=json&geocode='
    r = requests.get(api_url+adress)
    d = json.loads(r.text)
    if d['response']['GeoObjectCollection']['metaDataProperty']['GeocoderResponseMetaData']['found'] != 0:
        longitude, latitude = d['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split()
        return (latitude, longitude)
    return tuple()


def f(row):
    try:
        adress = row['adress'].replace(' ', '+')
        coordinates = get_lat_lon(adress)
        if coordinates:
            return ([row['url'], row['adress'], ', '.join([str(c) for c in coordinates]), row['salary']])
    except urllib.error.HTTPError:
        print(url)
    except IndexError:
        pass

def save_rows_as_txt(city_abbr):

    rows = [row for index, row in read_df(city_abbr).iterrows()]
    row_chunks = chunks(rows, 2)


    for chunk in row_chunks:

        pool = ThreadPool(8) 

        results = pool.map(f, chunk)

        pool.close() 
        pool.join()
        
        print(results)

        with open(f"../hh-data/tables/{city_abbr}/fulltime/rows_with_lat_lon.txt", "a") as file:
            file.write('\n'.join(['; '.join([str(el) for el in row]) for row in r]))


In [69]:
r = save_rows_as_txt('spb')

[['https://spb.hh.ru/vacancy/30707536', 'Пролетарская, Санкт-Петербург, проспект Обуховской Обороны, 112к2И', '59.866635, 30.472022', 4], ['https://spb.hh.ru/vacancy/30539353', 'Московская, Санкт-Петербург, 2 линия, метро Московская', '59.852018, 30.322399', 8]]


In [139]:
def write_as_csv(city_abbr):
    with open(f"../hh-data/tables/{city_abbr}/fulltime/rows_with_lat_lon.txt") as file:
        rows = [line.split('; ') for line in file.read().replace("https", "\nhttps").split('\n')]
    rows = [row for row in rows if len(row) == 4]
    df = pd.DataFrame(rows)
    df.columns = ['url', 'adress', 'coordinates', 'salary']
    df.to_csv(f"../hh-data/tables/{city_abbr}/fulltime/url-adress-coord-salary.csv", index=False)
    return df

In [142]:
rs = write_as_csv('kzn')

,url,adress,coordinates,salary
0,https://spb.hh.ru/vacancy/30707536,"Пролетарская, Санкт-Петербург, проспект Обухов...","59.866635, 30.472022",4
1,https://spb.hh.ru/vacancy/30539353,"Московская, Санкт-Петербург, 2 линия, метро Мо...","59.852018, 30.322399",8
2,https://spb.hh.ru/vacancy/30388832,"Санкт-Петербург, Гостиный двор,","59.934064, 30.332756",9
3,https://spb.hh.ru/vacancy/30673196,"Санкт-Петербург, Проспект Ветеранов,","59.832995, 30.191407",9
4,https://spb.hh.ru/vacancy/30722524,"Звёздная, Санкт-Петербург, Пулковское шоссе, 25к1","59.819802, 30.317252",10
5,https://spb.hh.ru/vacancy/30407527,"Лесная, Санкт-Петербург, Кантемировская улица, 37","59.985638, 30.353579",11
6,https://spb.hh.ru/vacancy/30508993,"Спортивная, Санкт-Петербург, 3-я линия Василье...","59.948816, 30.278094",11
7,https://spb.hh.ru/vacancy/30586230,"Василеостровская, Санкт-Петербург, 8-я линия В...","59.937301, 30.282406",12
8,https://spb.hh.ru/vacancy/30537883,"Международная, Санкт-Петербург, Будапештская у...","59.869065, 30.368563",12
9,https://spb.hh.ru/vacancy/30584960,"Московские ворота, Санкт-Петербург, Московский...","59.889882, 30.318734",12


In [133]:
df

,0,1,2,3
0,https://spb.hh.ru/vacancy/30707536,"Пролетарская, Санкт-Петербург, проспект Обухов...","59.866635, 30.472022",4
1,https://spb.hh.ru/vacancy/30539353,"Московская, Санкт-Петербург, 2 линия, метро Мо...","59.852018, 30.322399",8
2,https://spb.hh.ru/vacancy/30388832,"Санкт-Петербург, Гостиный двор,","59.934064, 30.332756",9
3,https://spb.hh.ru/vacancy/30673196,"Санкт-Петербург, Проспект Ветеранов,","59.832995, 30.191407",9
4,https://spb.hh.ru/vacancy/30722524,"Звёздная, Санкт-Петербург, Пулковское шоссе, 25к1","59.819802, 30.317252",10
5,https://spb.hh.ru/vacancy/30407527,"Лесная, Санкт-Петербург, Кантемировская улица, 37","59.985638, 30.353579",11
6,https://spb.hh.ru/vacancy/30508993,"Спортивная, Санкт-Петербург, 3-я линия Василье...","59.948816, 30.278094",11
7,https://spb.hh.ru/vacancy/30586230,"Василеостровская, Санкт-Петербург, 8-я линия В...","59.937301, 30.282406",12
8,https://spb.hh.ru/vacancy/30537883,"Международная, Санкт-Петербург, Будапештская у...","59.869065, 30.368563",12
9,https://spb.hh.ru/vacancy/30584960,"Московские ворота, Санкт-Петербург, Московский...","59.889882, 30.318734",12
